## Cython orderbook routines 

In [1]:
import cython
%load_ext cython

In [2]:
%%cython -a
# distutils: language=c++

from libcpp.map cimport map
from libcpp.pair cimport pair
from cython.operator cimport dereference

ctypedef map[int, int] TOrders
ctypedef pair[int, int] TOrder

ctypedef map[float, TOrders] TBook
ctypedef pair[float, TOrders] TBookItem

cdef TBook Book

cpdef existPrice(key: float):
    return Book.find(key) != Book.end()

cpdef Insert(float price, int orderID, int amt):
    """Insert the new order into orderbook"""
    cdef TOrders Orders
    
    if existPrice(price):
        Book[price].insert(TOrder(orderID, amt))
    else:
        Orders.insert(TOrder(orderID, amt))
        Book.insert(TBookItem(price, Orders))
    
cpdef Delete(float price, int orderID):
    """Delete one order from orderbook"""
    if existPrice(price):
        Book[price].erase(orderID)
        if Book[price].size() == 0:
            Book.erase(price)
        
    
def getBook():
    return Book

def firstItem():
    item = dereference(Book.begin()) if not Book.empty() else None
    return item

def firstPrice():
    return dereference(Book.begin()).first if not Book.empty() else 0

def lastItem():
    return dereference(Book.rbegin()) if not Book.empty() else None

def lastPrice():
    return dereference(Book.rbegin()).first if not Book.empty() else 0


In [3]:
%timeit existPrice(1000)
existPrice(12)

73.4 ns ± 1.61 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


False

In [4]:
Insert(1000, 12, 10)
getBook()

{1000.0: {12: 10}}

In [5]:
Delete(1000, 12)
getBook()

{}

In [6]:
Insert(1001.25, 14, 10)
getBook()

{1001.25: {14: 10}}

In [7]:
%timeit Insert(1001.25, 15, 100)
getBook()

168 ns ± 2.58 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


{1001.25: {14: 10, 15: 100}}

In [8]:
%timeit existPrice(1000)
existPrice(1000)

75.4 ns ± 1.23 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


False

In [9]:
Delete(1001.25, 14)
getBook()

{1001.25: {15: 100}}

In [10]:
%timeit firstItem()
firstItem()

255 ns ± 3.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


(1001.25, {15: 100})

In [11]:
%timeit lastItem()
lastItem()

252 ns ± 0.825 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


(1001.25, {15: 100})

In [12]:
%timeit getBook()
getBook()

285 ns ± 3.46 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


{1001.25: {15: 100}}